<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Using ChatGPT as a writing assistant of English for Research Publication Purposes (ERPP)

## Importing the required libraries

In [1]:
import openai
import pandas as pd
from os import environ
import datetime as dt
from google.cloud import translate
from IPython.display import clear_output

## Setting the required environment variables in the environment

Please refer to [Setting environment variables](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#setting-environment-variables) on [Managing environments](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#).

Note: Do **not** place the value of the variables between ' ' or " ".

In [ ]:
(base) C:\Users\eyamr>conda env list
# conda environments:
#
base                  *  C:\Users\eyamr\anaconda3
Env20240401              C:\Users\eyamr\anaconda3\envs\Env20240401


(base) C:\Users\eyamr>conda activate Env20240401

(Env20240401) C:\Users\eyamr>conda env config vars list

(Env20240401) C:\Users\eyamr>conda env config vars set GOOGLE_CLOUD_PROJECT=<omitted>
To make your changes take effect please reactivate your environment

(Env20240401) C:\Users\eyamr>conda env config vars set OPENAI_API_KEY=<omitted>
To make your changes take effect please reactivate your environment

(Env20240401) C:\Users\eyamr>conda activate Env20240401

(Env20240401) C:\Users\eyamr>conda env config vars list
GOOGLE_CLOUD_PROJECT = <omitted>
OPENAI_API_KEY = <omitted>

(Env20240401) C:\Users\eyamr>conda deactivate

(base) C:\Users\eyamr>

## Importing the required programme variables from the environment

In [2]:
openai.api_key = environ.get('OPENAI_API_KEY', '')
assert openai.api_key
GOOGLE_CLOUD_PROJECT = environ.get('GOOGLE_CLOUD_PROJECT', '')
assert GOOGLE_CLOUD_PROJECT
GOOGLE_CLOUD_PROJECT = str(GOOGLE_CLOUD_PROJECT)
PARENT = f'projects/{GOOGLE_CLOUD_PROJECT}'
#print(openai.api_key)
#print(GOOGLE_CLOUD_PROJECT)
#print(PARENT)

## Defining a function to translate passages with Google Cloud Translation API

In [3]:
def translate_text(text: str, target_language_code: str) -> translate.Translation:
    client = translate.TranslationServiceClient()
    response = client.translate_text(
        parent = PARENT,
        contents = [text],
        target_language_code = target_language_code
    )
    return response.translations[0]

## Defining a function to query ChatGPT

In [4]:
def get_completion(prompt, model = 'gpt-3.5-turbo'):
    client = openai.OpenAI()
    messages = [{'role': 'user', 'content': prompt}]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    return response.choices[0].message.content

## Collecting input

In [5]:
end = False
while end == False:
    filename = str(input('Enter the input full filename: '))
    if filename != '':
        try:
            with open(filename, 'r', encoding = 'utf8', newline='\n') as responses:
                print('The file exists.')
            input_file = filename
            output_file = input_file + '.out.txt'
            output_file_json = input_file + '.out.json'
            output_file_excel = input_file + '.out.xlsx'
            end = True
            clear_output()
        except FileNotFoundError:
            print('No such file.')
df_text = pd.read_table(input_file, sep = '\\n', header = None, engine = 'python')
df_text = df_text.rename(columns = {0: 'notes'})
df_text.at[0, 'composed'] = 1
df_text = df_text.astype('object')
print(str(len(df_text)) + ' passages to process.')

1 passages to process.


In [ ]:
df_text.dtypes

## Prompting ChatGPT

### Prompt

Dear ChatGPT, please write a piece of academic text based on the following notes considering the generally accepted standards of English for Academic Purposes. It is very important that you are as objective, scientific and non-metaphorical as you can be.\n

In [6]:
with open(output_file, 'a', encoding = 'utf8', newline='\n') as responses:
    responses.write('ChatGPT ERPP writing assistant' + '\n\n')
    responses.write('Start time: ' + str(dt.datetime.now()) + '\n\n')
    prompt = 'Dear ChatGPT, please write a piece of academic text based on the following notes considering the generally accepted standards of English for Academic Purposes. It is very important that you are as objective, scientific and non-metaphorical as you can be.\n'
    for index, row in df_text.iterrows():
        responses.write('Notes ' + str(index) + ':\n' + row['notes'] + '\n\n')
        print('Notes ' + str(index) + ':\n' + row['notes'])
        query = get_completion(prompt + row['notes'])
        df_text.at[index, 'composed'] = query
        responses.write('Composed ' + str(index) + ':\n' + query + '\n\n')
        print('\nComposed ' + str(index) + ':\n' + query)
        clear_output(wait = True)
    responses.write('End time: ' + str(dt.datetime.now()) + '\n\n')
print('Job completed!')
df_text.to_json(output_file_json)
writer = pd.ExcelWriter(output_file_excel)
df_text.to_excel(writer, sheet_name = 'text')
writer.close()
df_text.head()

Job completed!


,notes,composed
0,"In the history of Corpus Linguists, a number o...","In the field of Corpus Linguistics, various sc..."
